In [1]:
import os
from cnnClassifier.utils.common import read_yaml,create_directories

In [ ]:
import os
import shutil
import random
from tqdm import tqdm
images_des="artifacts\data_ingestion\Train\\"
data_des="artifacts\Processed_data\\"
split="train\\"
for l in tqdm(os.listdir(images_des)):
    clas=l.split(".")[0]
    if random.random()<0.8:
        split="train\\"
    else:
        split="test\\"
    copy_des=data_des+split+clas


    if not os.path.exists(copy_des):
        # Create the folder
        os.makedirs(copy_des)
        print(f"Folder '{copy_des}' created successfully.")


    try :
        shutil.move(images_des+l,copy_des)
    except:
        print("faced_error")


In [2]:
os.chdir("..")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataStructureConfig:
    unprocessed_data:Path
    processed_data : Path
    train_folder_sal : Path
    test_folder_sal :Path
    train_folder_cocc : Path
    test_folder_cocc :Path



In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [5]:
CONFIG_FILE_PATH=Path("config/config.yaml")
PARAMS_FILE_PATH=Path("params.yaml")

In [6]:
class DataIngestionConfig:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        # self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def proper_dataStructure_config(self) -> DataStructureConfig:
        config = self.config.data_structure
        create_directories([config.processed_data,config.train_folder_sal,config.test_folder_sal,config.train_folder_cocc,config.test_folder_cocc])
        data_structure_config = DataStructureConfig(
            unprocessed_data=config.unprocessed_data,
            processed_data=config.processed_data,
            train_folder_sal =config.train_folder_sal,
            test_folder_sal =config.test_folder_sal,
            train_folder_cocc =config.train_folder_cocc,
            test_folder_cocc =config.test_folder_cocc
        )
        return data_structure_config
    

In [7]:
%pwd

'c:\\Users\\atanu\\Desktop\\MY_WORK\\mlops\\chicken_disease_classification'

In [8]:
config=DataIngestionConfig()
data_structure_config=config.proper_dataStructure_config()

[2023-12-01 19:00:50,300 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-12-01 19:00:50,301 : INFO : common : created directory at: artifacts]
[2023-12-01 19:00:50,302 : INFO : common : created directory at: artifacts/processed_data]
[2023-12-01 19:00:50,303 : INFO : common : created directory at: artifacts/processed_data/train/Salmonella]
[2023-12-01 19:00:50,304 : INFO : common : created directory at: artifacts/processed_data/test/Salmonella]
[2023-12-01 19:00:50,305 : INFO : common : created directory at: artifacts/processed_data/train/Coccidiosis]
[2023-12-01 19:00:50,306 : INFO : common : created directory at: artifacts/processed_data/test/Coccidiosis]


In [17]:
import random
random.random()

0.44851665739483226

 12%|█▏        | 359/2966 [00:00<00:01, 1792.97it/s]

Folder 'artifacts\Processed_data\test\healthy' created successfully.
Folder 'artifacts\Processed_data\train\healthy' created successfully.


 76%|███████▌  | 2261/2966 [00:01<00:00, 1382.90it/s]

Folder 'artifacts\Processed_data\train\ncd' created successfully.
Folder 'artifacts\Processed_data\test\ncd' created successfully.


 91%|█████████ | 2694/2966 [00:01<00:00, 1412.22it/s]

Folder 'artifacts\Processed_data\test\pcrhealthy' created successfully.
Folder 'artifacts\Processed_data\train\pcrhealthy' created successfully.


100%|██████████| 2966/2966 [00:02<00:00, 1464.29it/s]

Folder 'artifacts\Processed_data\test\pcrncd' created successfully.
Folder 'artifacts\Processed_data\train\pcrncd' created successfully.


In [9]:
import os
import zipfile
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        os.environ['KAGGLE_CONFIG_DIR'] =self.config.kaggle_path
       
        dataset_slug = self.config.kaggle_data
        destination_folder = self.config.local_data_file
        data_name=destination_folder+dataset_slug.split('/')[-1]+".zip"
        print(not os.path.exists(data_name))
        if not os.path.exists(data_name):
            # Create the command to download the dataset into the specified folder

            command = f"kaggle datasets download -d {dataset_slug} -p {destination_folder}"
            print(command)
            # Run the download command
            os.system(command)
            logger.info(f"{dataset_slug} downloaded ! ")
        else:
            logger.info(f"The file is already exists of size : {get_size(data_name)}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        dataset_slug = self.config.kaggle_data
        path_dir=unzip_path+dataset_slug.split('/')[-1]+".zip"
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(path_dir, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"file has been successfully exxtracted .")
        except:
            logger.info("there is some error while extracting the files")




In [10]:
%pwd

'c:\\Users\\atanu\\Desktop\\MY_WORK\\mlops\\chicken_disease_classification'

In [11]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
class ProcessingData:
    def __init__(self,config : DataStructureConfig):
        self.config=config

    def split_data(self):
        csv_file=self.config.unprocessed_data +"/train_data.csv"
        data_file=pd.read_csv(csv_file)
        train,test =train_test_split(data_file,test_size=0.2)
        return train,test

    def process_data(self):
        train,test=self.split_data()
        images_folder=self.config.unprocessed_data+"/Train/"
        train_folder=self.config.processed_data+"/train/"
        test_folder=self.config.processed_data+"/test/"



        # Iterate through the each row in the train dataframe

        for index , row in tqdm(train.iterrows()):
            try:
                image_name=row["images"]
                class_name=row["label"]

                # Move the image to the corresponding class folder
                source_image_path = os.path.join(images_folder, image_name)
                destination_image_path = os.path.join(train_folder, class_name+"/"+image_name)

                # Check if the image file exists before moving
                if os.path.exists(source_image_path):
                    shutil.move(source_image_path, destination_image_path)

                else:
                    print(f"Image '{image_name}' not found in the images folder.")
            except:
                print("Faced error while moving image")

        
        # Iterate through the each row in the train dataframe
        ct=0
        for index , row in tqdm(test.iterrows()):
            try:
                image_name=row["images"]
                class_name=row["label"]

                # Move the image to the corresponding class folder
                source_image_path = os.path.join(images_folder, image_name)
                destination_image_path = os.path.join(test_folder, class_name+"/"+image_name)

                # Check if the image file exists before moving
                if os.path.exists(source_image_path):
                    shutil.move(source_image_path, destination_image_path)
                    ct+=1
                else:
                    print(f"Image '{image_name}' not found in the images folder.")
            except:
                print("Faced error while moving image")

            print(ct)


In [12]:
# import pandas as pd
# import os
# import shutil

# # Read the CSV file into a pandas DataFrame
# csv_file = 'path/to/your/csv_file.csv'  # Replace with your CSV file path
# data = pd.read_csv(csv_file)

# # Path to the folder containing the images
# images_folder = 'path/to/your/images_folder/'  # Replace with your images folder path

# # Iterate through each row in the DataFrame
# for index, row in data.iterrows():
#     image_name = row['image_name']
#     class_name = row['class']

#     # Create folders for classes if they don't exist
#     class_folder = os.path.join(images_folder, class_name)
#     os.makedirs(class_folder, exist_ok=True)

#     # Move the image to the corresponding class folder
#     source_image_path = os.path.join(images_folder, image_name)
#     destination_image_path = os.path.join(class_folder, image_name)

#     # Check if the image file exists before moving
#     if os.path.exists(source_image_path):
#         shutil.move(source_image_path, destination_image_path)
#     else:
#         print(f"Image '{image_name}' not found in the images folder.")

#     print("Images moved to corresponding class folders.")


In [13]:
p=ProcessingData(data_structure_config)


In [14]:
p.process_data()

0it [00:00, ?it/s]

992it [00:00, 4997.61it/s]

Faced error while moving image
Image 'cocci.1119.jpg' not found in the images folder.
Image 'salmo.1839.jpg' not found in the images folder.
Image 'cocci.2011.jpg' not found in the images folder.
Image 'pcrsalmo.303.jpg' not found in the images folder.
Faced error while moving image
Faced error while moving image
Image 'pcrcocci.83.jpg' not found in the images folder.
Faced error while moving image
Faced error while moving image
Faced error while moving image
Image 'salmo.711.jpg' not found in the images folder.
Faced error while moving image
Image 'cocci.127.jpg' not found in the images folder.
Image 'cocci.1069.jpg' not found in the images folder.
Image 'salmo.658.jpg' not found in the images folder.
Faced error while moving image
Image 'salmo.1784.jpg' not found in the images folder.
Image 'salmo.2081.jpg' not found in the images folder.
Faced error while moving image
Faced error while moving image
Faced error while moving image
Image 'cocci.1587.jpg' not found in the images folder.

2037it [00:00, 5144.51it/s]

Image 'pcrcocci.298.jpg' not found in the images folder.
Image 'pcrcocci.246.jpg' not found in the images folder.
Image 'salmo.1188.jpg' not found in the images folder.
Faced error while moving image
Faced error while moving image
Image 'salmo.267.jpg' not found in the images folder.
Image 'pcrsalmo.130.jpg' not found in the images folder.
Image 'salmo.1873.jpg' not found in the images folder.
Image 'cocci.261.jpg' not found in the images folder.
Image 'pcrsalmo.266.jpg' not found in the images folder.
Image 'cocci.832.jpg' not found in the images folder.
Faced error while moving image
Faced error while moving image
Image 'salmo.1750.jpg' not found in the images folder.
Image 'pcrsalmo.4.jpg' not found in the images folder.
Faced error while moving image
Image 'pcrsalmo.101.jpg' not found in the images folder.
Image 'cocci.753.jpg' not found in the images folder.
Image 'cocci.331.jpg' not found in the images folder.
Faced error while moving image
Image 'cocci.1441.jpg' not found in the

...

Faced error while moving image
0
Image 'salmo.1044.jpg' not found in the images folder.
0
Image 'salmo.228.jpg' not found in the images folder.
0
Image 'cocci.1193.jpg' not found in the images folder.
0
Image 'salmo.1787.jpg' not found in the images folder.
0
Image 'cocci.174.jpg' not found in the images folder.
0
Image 'cocci.988.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'pcrcocci.188.jpg' not found in the images folder.
0
Faced error while moving image
0
Faced error while moving image
0
Faced error while moving image
0
Image 'salmo.1127.jpg' not found in the images folder.
0
Image 'cocci.1214.jpg' not found in the images folder.
0
Image 'salmo.59.jpg' not found in the images folder.
0
Image 'salmo.636.jpg' not found in the images folder.
0
Image 'cocci.1735.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'cocci.1730.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'cocci.768.jpg' not found 

1005it [00:00, 5021.91it/s]

Faced error while moving image
0
Faced error while moving image
0
Image 'salmo.2234.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'salmo.2005.jpg' not found in the images folder.
0
Image 'cocci.257.jpg' not found in the images folder.
0
Image 'cocci.774.jpg' not found in the images folder.
0
Image 'salmo.2208.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'cocci.1351.jpg' not found in the images folder.
0
Image 'salmo.1299.jpg' not found in the images folder.
0
Image 'salmo.1847.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'salmo.1423.jpg' not found in the images folder.
0
Image 'salmo.818.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'cocci.1801.jpg' not found in the images folder.
0
Image 'cocci.14.jpg' not found in the images folder.
0
Image 'pcrsalmo.25.jpg' not found in the images folder.
0
Image 'pcrcocci.172.jpg' not found in the images folder.
0
Image 

1614it [00:00, 4664.16it/s]

Faced error while moving image
0
Image 'salmo.280.jpg' not found in the images folder.
0
Image 'cocci.1366.jpg' not found in the images folder.
0
Image 'pcrsalmo.118.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'salmo.2178.jpg' not found in the images folder.
0
Image 'salmo.1312.jpg' not found in the images folder.
0
Image 'cocci.1928.jpg' not found in the images folder.
0
Faced error while moving image
0
Faced error while moving image
0
Faced error while moving image
0
Image 'pcrsalmo.261.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'pcrcocci.158.jpg' not found in the images folder.
0
Image 'salmo.1739.jpg' not found in the images folder.
0
Faced error while moving image
0
Faced error while moving image
0
Image 'salmo.2049.jpg' not found in the images folder.
0
Image 'cocci.1113.jpg' not found in the images folder.
0
Faced error while moving image
0
Image 'salmo.610.jpg' not found in the images folder.
0
Image 'pcrsalmo

In [15]:
f=pd.read_csv("artifacts/data_ingestion/train_data.csv")

In [98]:
f.head()

,images,label
0,salmo.1558.jpg,Salmonella
1,cocci.1866.jpg,Coccidiosis
2,cocci.171.jpg,Coccidiosis
3,salmo.1484.jpg,Salmonella
4,ncd.100.jpg,New Castle Disease


In [90]:
! pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/1c/49/30ffcac5af06d08dfdd27da322ce31a373b733711bb272941877c1e4794a/scikit_learn-1.3.2-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB 320.0 kB/s eta 0:00:30
   ---------------------------------------- 0.1/9.3 MB 1.0 MB/s eta 0:00:09
   - -------------------------------------- 0.3/9.3 MB 2.5 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/9.3 MB 4.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.3 MB 4.6 MB/s eta 0:00:02
   ----- ---------------------------------- 